In [ ]:
# Настройка импортов
import pandas as pd
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from umap import UMAP
from bertopic import BERTopic
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import plotly.express as px
import logging
import optuna
from functools import partial
import os
import zipfile
import warnings
import re
from llama_cpp import Llama

In [ ]:
warnings.filterwarnings("ignore", message="The default value of `min_cluster_size` has changed from 5 to 10. This will affect the number of clusters produced.")
warnings.filterwarnings("ignore", message="The default value of `min_samples` has changed from None to `min_cluster_size`. This will affect the number of clusters produced.")

# Настройка стандартного сообщения лога
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Загрузка SpaCy модели
try:
    nlp = spacy.load("en_core_web_lg")
    logging.info("SpaCy model 'en_core_web_lg' successfully loaded.") #
except OSError:
    logging.error("SpaCy model 'en_core_web_lg' not found. Please install it with: python -m spacy download en_core_web_lg")
    exit()

In [ ]:
# Функция загрузки входных данных из файла
def process_texts_from_file(file_path: str, nlp_model):
    logging.info(f"Starting preprocessing of text requirements from file: {file_path}...") #
    requirements = []
    processed_texts = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f: #
            for line in f:
                line = line.strip() # Удаление пробелов в начале/конце строки
                if line:
                    requirements.append(line) #
                    processed_texts.append(preprocess_text(line, nlp_model))
        logging.info(f"Preprocessing completed. Processed {len(requirements)} requirements.")
    except FileNotFoundError:
        logging.error(f"File not found at path: {file_path}")
        return [], []
    except Exception as e:
        logging.error(f"Error reading or processing file: {e}")
        return [], []
    return requirements, processed_texts

In [ ]:
# Функция пред обработки входных данных
def preprocess_text(text: str, nlp_model) -> str:
    stopwords = set(nlp_model.Defaults.stop_words)
    doc = nlp_model(text.lower())
    tokens = []
    for token in doc:
        # Удаление стоп слов, пунктуации и пробелов и не алфавитных символов
        if not token.is_punct and not token.is_space and not token.is_stop and token.text not in stopwords and token.is_alpha: #
            if re.search(r'[a-zA-Z]', token.lemma_):
                tokens.append(token.lemma_)
    return " ".join(tokens)

In [ ]:
#Блок загрузки исходных данных
data_path = "tz_erp.md"
original_requirements, processed_requirements = process_texts_from_file(data_path, nlp_model=nlp)

if not original_requirements:
    logging.error("No data to process. Exiting program.") #
    exit()
df_requirements = pd.DataFrame({
    'Requirement': original_requirements,
    'Processed_Requirement': processed_requirements
})
logging.info(f"Loaded and preprocessed {len(df_requirements)} requirements.") #

In [ ]:
# Функция генерации эмбеддингов
def generate_embeddings(texts: list, model_name: str = 'thenlper/gte-large'):
    logging.info(f"Loading SentenceTransformer model: {model_name}...") #
    try:
        model = SentenceTransformer(model_name) # Загрузка пред обученной модели, отвечающей за генерацию эмбеддингов
        logging.info("SentenceTransformer model successfully loaded.")
    except Exception as e:
        logging.error(f"Error loading SentenceTransformer model: {e}")
        raise
    logging.info("Starting embedding generation...")
    # Генерация эмбеддингов из обработанных входных данных
    embeddings = model.encode(texts, show_progress_bar=True)
    logging.info(f"Embedding generation completed. Dimensionality: {embeddings.shape}") #
    return embeddings

In [ ]:
# Блок с вызовом функции генерации эмбеддиногов
document_embeddings = generate_embeddings(df_requirements['Processed_Requirement'].tolist())

In [ ]:
# Функция оптимизации гиперпараметров моделей с использованием фреймворка Optuna
def objective(trial, documents, embeddings, nlp_model):
    # Гиперпараметры UMAP
    # Количество соседей для построения графа локальных связей.
    # Меньшие значения делают модель чувствительной к локальной структуре,
    # большие — к глобальной.
    n_neighbors = trial.suggest_int('umap_n_neighbors', 5, 20)
    # Размерность выходного пространства (число компонентов после уменьшения размерности).
    # Меньшие значения подходят для визуализации, большие сохраняют больше информации.
    n_components = trial.suggest_int('umap_n_components', 2, 7)
    # Минимальное расстояние между точками в результирующем пространстве.
    # Меньшие значения создают плотные кластеры, большие раздвигают их.
    min_dist = trial.suggest_float('umap_min_dist', 0.0, 0.3)
    # Гиперпараметры HDBSCAN
    # Минимальный размер кластера. Точки, не входящие в кластеры такого размера,
    # считаются шумом.
    min_cluster_size = trial.suggest_int('hdbscan_min_cluster_size', 5, 15)
    # Минимальное количество соседей для формирования плотного региона (core points).
    # Должно быть <= min_cluster_size.
    min_samples = trial.suggest_int('hdbscan_min_samples', 1, min_cluster_size - 1) # min_samples <= min_cluster_size
    if min_samples >= min_cluster_size:
        min_samples = min_cluster_size - 1
        if min_samples < 1: # Fallback for very small min_cluster_size
            min_samples = 1

    # Максимальное расстояние между точками в одном кластере.
    # Большие значения объединяют близкие кластеры
    cluster_selection_epsilon = trial.suggest_float('hdbscan_cluster_selection_epsilon', 0.0, 1.0)

    # Гиперпараметров CountVectorizer
    # Минимальная частота слова (в долях от общего числа документов),
    # при которой оно считается слишком распространенным и исключается из словаря.
    max_df = trial.suggest_float('vectorizer_max_df', 0.8, 1.0)
    # Максимальная длина n-грамм (например, униграммы, биграммы, триграммы).
    # Большие значения учитывают длинные комбинации слов.
    ngram_range_end = trial.suggest_int('vectorizer_ngram_range_end', 1, 4) #

    # Создание экземпляров моделей с предложенными гиперпараметрами
    umap_model = UMAP(
        n_neighbors=n_neighbors,  # Чувствительность к локальной/глобальной структуре
        n_components=n_components,  # Размерность выходного пространства
        min_dist=min_dist,  # Плотность кластеров
        metric='cosine',
        random_state=42,
        low_memory=False
    )

    hdbscan_model = HDBSCAN(
        min_cluster_size=min_cluster_size,  # Минимальный размер кластера
        min_samples=min_samples,  # Плотность регионов
        metric='euclidean',
        cluster_selection_epsilon=cluster_selection_epsilon,  # Расстояние между точками
        prediction_data=True
    )

    vectorizer_model = CountVectorizer(
        stop_words=list(nlp_model.Defaults.stop_words),  # Исключение стоп-слов
        min_df=1,  # Минимальная частота слова
        max_df=max_df,  # Максимальная частота слова
        ngram_range=(1, ngram_range_end)  # Учет n-грамм
    )

    # Инициализация BERTopic с выбранными моделями
    topic_model_opt = BERTopic(
        language="multilingual",
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        top_n_words=10,
        calculate_probabilities=True
    )

    try:
        # Обучение модели и получение тем/вероятностей
        topics, probabilities = topic_model_opt.fit_transform(documents, embeddings)

        labels = np.array(topics)
        unique_labels = np.unique(labels)
        num_topics = len(unique_labels[unique_labels != -1])
        # Если кластеров недостаточно для осмысленной оценки
        if num_topics < 2:
            logging.warning(f"Trial {trial.number}: Only {num_topics} non-noise topics found. Returning a poor score.")
            return -1.0
        # Создаем булеву маску для исключения шумовых точек (топик -1)
        # Метрики качества кластеризации (Silhouette, DB, CH) не предназначены для оценки шума,
        # поэтому расчет производится только на кластеризованных данных.
        non_noise_indices = (labels != -1)

        # Проверка: если все точки являются шумом или нет кластеризованных точек
        if not np.any(non_noise_indices):
            logging.warning(f"Trial {trial.number}: All points are noise. Returning a poor score.")
            return -1.0

        # Фильтруем эмбеддинги и метки, оставляя только те, что относятся к кластерам (не к шуму)
        filtered_embeddings = embeddings[non_noise_indices]
        filtered_labels = labels[non_noise_indices] # Используем labels вместо topics для консистентности

        if len(np.unique(filtered_labels)) < 2:
            logging.warning(f"Trial {trial.number}: Less than 2 unique non-noise topics after filtering. Returning a poor score.")
            return -1.0

        # Вычисление метрик качества кластеризации
        s_score = silhouette_score(filtered_embeddings, filtered_labels)
        db_score = davies_bouldin_score(filtered_embeddings, filtered_labels)
        ch_score = calinski_harabasz_score(filtered_embeddings, filtered_labels)

        # Целевая функция: максимизация силуэта и минимизация Davies-Bouldin
        objective_score = s_score - np.log1p(db_score) + np.log1p(ch_score)
        logging.info(f"Trial {trial.number}: Silhouette={s_score:.4f}, Davies-Bouldin={db_score:.4f}, Calinski-Harabasz={ch_score:.4f}, Score={objective_score:.4f}")
        return objective_score
    except Exception as e:
        logging.error(f"Trial {trial.number}: Error during training: {e}")
        score = -1.0 # Штраф за ошибку
    logging.info(f"Trial {trial.number}: UMAP(n_neighbors={n_neighbors}, n_components={n_components}, min_dist={min_dist}) | HDBSCAN(min_cluster_size={min_cluster_size}, min_samples={min_samples}, epsilon={cluster_selection_epsilon}) | Vectorizer(min_df={1}, max_df={max_df}, ngram={ngram_range_end}) -> Silhouette Score: {score:.4f}")
    return score

In [ ]:
# Функция, инициализирующая оптимизацию гиперпараметров моделей с заданым количеством этапов
def optimize_bertopic_hyperparameters(documents: list, embeddings: np.ndarray, nlp_model, n_trials: int = 50):
    logging.info(f"Starting BERTopic hyperparameter optimization with Optuna (n_trials={n_trials})...")
    objective_with_data = partial(objective, documents=documents, embeddings=embeddings, nlp_model=nlp_model)
    study = optuna.create_study(direction='maximize', study_name='BERTopic_Optimization')
    study.optimize(objective_with_data, n_trials=n_trials, show_progress_bar=True)
    logging.info(f"Optimization completed. Best parameters: {study.best_params}")
    logging.info(f"Best Silhouette Score: {study.best_value:.4f}")
    return study.best_params

In [ ]:
# Получение оптимальных параметров
best_params = optimize_bertopic_hyperparameters(df_requirements['Processed_Requirement'].tolist(), document_embeddings, nlp_model=nlp, n_trials=150)

In [ ]:
# Функция обучения финальной модели BERTopic
def train_bertopic_model(documents: list, embeddings: np.ndarray, best_params: dict, nlp_model):
    logging.info("\nInitializing final BERTopic model with optimized parameters...")

    # Инициализация UMAP-модели с оптимизированными гиперпараметрами
    final_umap_model = UMAP(
        n_neighbors=best_params['umap_n_neighbors'],  # Количество соседей для построения графа локальных связей.
                                                       # Меньшие значения делают модель чувствительной к локальной структуре,
                                                       # большие — к глобальной.
        n_components=best_params['umap_n_components'],  # Размерность выходного пространства (число компонентов после уменьшения размерности).
                                                         # Меньшие значения подходят для визуализации, большие сохраняют больше информации.
        min_dist=best_params['umap_min_dist'],  # Минимальное расстояние между точками в результирующем пространстве.
                                                 # Меньшие значения создают плотные кластеры, большие раздвигают их.
        metric='cosine',  # Метрика для измерения расстояния между точками (косинусное расстояние).
        random_state=42  # Фиксация случайного состояния для воспроизводимости результатов.
    )

    # Инициализация HDBSCAN-модели с оптимизированными гиперпараметрами
    final_hdbscan_model = HDBSCAN(
        min_cluster_size=best_params['hdbscan_min_cluster_size'],  # Минимальный размер кластера.
                                                                     # Точки, не входящие в кластеры такого размера, считаются шумом.
        min_samples=best_params['hdbscan_min_samples'],  # Минимальное количество соседей для формирования плотного региона (core points).
                                                           # Должно быть <= min_cluster_size.
        metric='euclidean',  # Метрика для измерения расстояния между точками (евклидово расстояние).
        cluster_selection_epsilon=best_params['hdbscan_cluster_selection_epsilon'],  # Максимальное расстояние между точками в одном кластере.
                                                                                        # Большие значения объединяют близкие кластеры.
        prediction_data=True  # Сохранение данных для предсказания новых точек.
    )

    # Инициализация CountVectorizer с оптимизированными гиперпараметрами
    final_vectorizer_model = CountVectorizer(
        stop_words=list(nlp_model.Defaults.stop_words),  # Исключение стоп-слов (например, "the", "and").
        min_df=1,  # Минимальная частота слова (в долях от общего числа документов).
                     # Слова, встречающиеся реже, исключаются из словаря.
        max_df=best_params['vectorizer_max_df'],  # Максимальная частота слова (в долях от общего числа документов).
                                                   # Слишком распространенные слова исключаются из словаря.
        ngram_range=(1, best_params['vectorizer_ngram_range_end'])  # Учет n-грамм (например, униграммы, биграммы, триграммы).
                                                                      # Большие значения учитывают длинные комбинации слов.
    )

    # Инициализация финальной модели BERTopic с оптимизированными моделями
    final_topic_model = BERTopic(
        language="multilingual",  # Поддержка мультиязычных текстов.
        umap_model=final_umap_model,  # Модель UMAP для уменьшения размерности данных.
        hdbscan_model=final_hdbscan_model,  # Модель HDBSCAN для кластеризации данных.
        vectorizer_model=final_vectorizer_model,  # Модель CountVectorizer для преобразования текста в числовые векторы.
        top_n_words=10,  # Количество топовых слов для описания каждой темы.
        calculate_probabilities=True  # Расчет вероятностей принадлежности текстов к темам.
    )

    # Обучение финальной модели BERTopic на документах и эмбеддингах
    topics, probabilities = final_topic_model.fit_transform(documents, embeddings)
    logging.info(f"Final BERTopic model training completed. Found topics: {len(final_topic_model.get_topics()) - 1}")
    # -1 для исключения шума (топик -1).

    return final_topic_model, topics, probabilities

In [ ]:
# Блок получения экземпляра обученной на оптимизированных гиперпараметрах модели
final_topic_model, topics, probabilities = train_bertopic_model(df_requirements['Processed_Requirement'].tolist(), document_embeddings, best_params, nlp)

# Добавление результатов в DataFrame
df_results = df_requirements.copy()
df_results['Embeddings'] = document_embeddings.tolist()
df_results['Topic'] = topics
df_results['Topic_Probability'] = [p.max() if p is not None and len(p) > 0 else 0 for p in probabilities]
df_results

In [ ]:
# Расчет финальных метрик качества разбиения
def evaluate_clustering(embeddings: np.ndarray, labels: np.ndarray):
    metrics = {}
    logging.info("\nStarting clustering quality evaluation for the final model...")

    # Удаление всех данных, помеченных как шум
    non_noise_indices = labels != -1
    filtered_embeddings = embeddings[non_noise_indices]
    filtered_labels = labels[non_noise_indices]

    if len(set(filtered_labels)) < 2:
        logging.warning("Insufficient clusters (less than 2, excluding noise) for calculating clustering quality metrics. Skipping metrics.") #
        metrics['Silhouette Score'] = np.nan
        metrics['Davies-Bouldin Index'] = np.nan
        metrics['Calinski-Harabasz Index'] = np.nan
        return metrics

    try:
        # Расчет коэффициента силуэта, значения от -1 до 1.
        metrics['Silhouette Score'] = silhouette_score(filtered_embeddings, filtered_labels)
    except Exception as e:
        logging.error(f"Error calculating Silhouette Score: {e}")
        metrics['Silhouette Score'] = np.nan

    try:
        # Расчет коэффициента Дэвиса-Боулдина, чем ниже - тем лучше. Минимальное значение - 0
        metrics['Davies-Bouldin Index'] = davies_bouldin_score(filtered_embeddings, filtered_labels)
    except Exception as e:
        logging.error(f"Error calculating Davies-Bouldin Index: {e}")
        metrics['Davies-Bouldin Index'] = np.nan

    try:
        # Расчет коэффициента Калински-Харабза: Чем выше - тем лучше, минимальное значение 0
        metrics['Calinski-Harabasz Index'] = calinski_harabasz_score(filtered_embeddings, filtered_labels)
    except Exception as e:
        logging.error(f"Error calculating Calinski-Harabasz Index: {e}")
        metrics['Calinski-Harabasz Index'] = np.nan

    logging.info("Clustering evaluation completed.")
    return metrics

In [ ]:
# Блок получения резульатов финальной кластеризации
clustering_metrics = evaluate_clustering(document_embeddings, np.array(topics))
logging.info("Clustering metrics for the final model:")
for metric, value in clustering_metrics.items():
    logging.info(f"- {metric}: {value:.4f}" if isinstance(value, float) else f"- {metric}: {value}")

In [ ]:
# Функция визуализации полученных результатов
def visualize_clusters(df_results: pd.DataFrame, umap_model: UMAP, title: str = "Requirement Clusters Visualization", dim: int = 2):
    # Будет построено 2 графика с отображнием на них получившихся кластеров, в 2D и в 3D
    logging.info(f"Starting {dim}D cluster visualization...")
    if 'Embeddings' not in df_results.columns or 'Topic' not in df_results.columns:
        logging.error("DataFrame for visualization must contain 'Embeddings' and 'Topic' columns.")
        return
    embeddings_to_project = np.array(df_results['Embeddings'].tolist())
    if umap_model is None:
        logging.error("UMAP model not provided for visualization.")
        return
    if dim == 2:
        if umap_model.n_components != 2:
            logging.warning("UMAP model initialized not for 2D. Recreating UMAP for 2D visualization.")
            umap_model_2d = UMAP(n_neighbors=umap_model.n_neighbors, n_components=2, min_dist=umap_model.min_dist, metric=umap_model.metric, random_state=umap_model.random_state)
            reduced_embeddings = umap_model_2d.fit_transform(embeddings_to_project)
        else:
            reduced_embeddings = umap_model.fit_transform(embeddings_to_project)
        df_results['UMAP_X'] = reduced_embeddings[:, 0]
        df_results['UMAP_Y'] = reduced_embeddings[:, 1]
        fig = px.scatter(df_results,x='UMAP_X',y='UMAP_Y',color='Topic',hover_data=['Requirement'],title=title,labels={'UMAP_X': 'UMAP Component 1', 'UMAP_Y': 'UMAP Component 2'})
    elif dim == 3:
        if umap_model.n_components != 3:
            logging.warning("UMAP model initialized not for 3D. Recreating UMAP for 3D visualization.")
            umap_model_3d = UMAP(n_neighbors=umap_model.n_neighbors, n_components=3, min_dist=umap_model.min_dist, metric=umap_model.metric, random_state=umap_model.random_state)
            reduced_embeddings = umap_model_3d.fit_transform(embeddings_to_project)
        else:
            reduced_embeddings = umap_model.fit_transform(embeddings_to_project)
        df_results['UMAP_X'] = reduced_embeddings[:, 0]
        df_results['UMAP_Y'] = reduced_embeddings[:, 1]
        df_results['UMAP_Z'] = reduced_embeddings[:, 2]

        fig = px.scatter_3d(df_results,x='UMAP_X',y='UMAP_Y',z='UMAP_Z',color='Topic',hover_data=['Requirement'],title=title,labels={'UMAP_X': 'UMAP Component 1','UMAP_Y': 'UMAP Component 2','UMAP_Z': 'UMAP Component 3' })
    else:
        logging.error("Unsupported dimension for visualization. Only 2D and 3D are supported.")
        return
    fig.show()
    logging.info(f"{dim}D Visualization completed.")

In [ ]:
# Блок вызова построения 2D и 3D графиков
visualize_clusters(df_results, final_topic_model.umap_model, title="BERTopic: Requirement Clusters Visualization (2D Optimized)", dim=2)
visualize_clusters(df_results, final_topic_model.umap_model, title="BERTopic: Requirement Clusters Visualization (3D Optimized)", dim=3)

In [ ]:
try:
    llm = Llama.from_pretrained(
	    repo_id="MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF",
	    filename="Mistral-7B-Instruct-v0.3.IQ1_M.gguf",
        n_gpu_layers=-1, # Использует все слои на GPU (Apple Metal)
        n_ctx=4096,      # Контекстное окно (можно увеличить, если нужно)
        chat_format="mistral-instruct", # Важно для корректного форматирования промптов
        verbose=False    # Отключить подробный вывод llama.cpp в консоль
    )
except Exception as e:
    logging.error(f"Ошибка при загрузке модели: {e}")

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)
# 1. Анализ количества уникальных кластеров
unique_topics = df_results['Topic'].unique()
num_unique_topics = len(unique_topics)
print(f"Общее количество уникальных кластеров (топиков): {num_unique_topics}")
print(f"Список уникальных топиков: {unique_topics}")

In [ ]:
# Группировка всех имеющихся кластеров в отдельные наборы данных
grouped_requirements = {}
for topic_id in unique_topics:
    # Исключаем топик -1, который обычно относится к шуму или неклассифицированным документам
    if topic_id == -1:
        continue
    requirements_in_topic = df_results[df_results['Topic'] == topic_id]['Processed_Requirement'].tolist()
    grouped_requirements[topic_id] = requirements_in_topic
    print(f"\nТопик {topic_id} содержит {len(requirements_in_topic)} требований.")
    print(f"Примеры требований в топике {topic_id}: {requirements_in_topic[:3]}...")

In [ ]:
# Функция, генерирующая подробное описание кластера на основе списка требований с помощью LLM.
def generate_system_description(requirements):
    requirements_str = "\n".join([f"- {req}" for req in requirements])
    prompt = \
f"""
Master prompt: Developing a detailed description of a software system based on functional requirements
Can you create a comprehensive and accurate description of a software system by following my detailed instructions? I expect you to generate a high-quality result suitable for use at an advanced level. If you are ready for this challenge in prompt engineering, get to work.
Goal: Based on the functional requirements provided, generate a complete, detailed, and structured description of the software system suitable for use at all stages of the development lifecycle — from conceptualization to deployment.
Required output format:
Your output should be extremely clear, concise, and organized, using the following structure:
1.  System name: A short but comprehensive name that accurately reflects the essence and main purpose of the system. It should be easy to remember and intuitive.
2.  Detailed description of the system:
A detailed but concise overview of the system. Answer the following questions:
What is the main purpose of the system? (The problem it solves or the value it creates).
Who is the target audience/users?
What are its key components and how do they interact? (A high-level overview of the architecture, without unnecessary technical details).
What are its main functions?
What are its expected benefits or outcomes?
In what environment will it operate? (E.g., web, desktop, mobile, cloud).
Use a coherent prose style to ensure smooth reading and deep understanding.
3.  Key functionalities:
A clear, bulleted list of all the main functionalities of the system that directly follow from the functional requirements provided.
Each item should be:
Specific: Describe one action or one capability.
Measurable: Allow determination of whether the feature is implemented.
Achievable: Realistic to implement within the system.
Relevant: Directly related to the system's objectives.
Time-bound: (Although this is not always applicable to individual features, keep the project context in mind).
Avoid redundancy and general phrases.
4.  Suggested high-level project directory structure:
Present a logical, intuitive, and scalable project directory structure. This structure should be general enough to apply to most modern software projects, but detailed enough to serve as a practical guide. Include, but don't limit yourself to, the following examples, if applicable and appropriate:
src/ (source code)
tests/ (tests: unit, integration, e2e)
docs/ (documentation: API, user guides, architecture)
Use a hierarchical representation (e.g., with indentation) for clarity.
Input: {requirements_str}
Processing instructions:
Analysis: Carefully analyze each functional requirement. Identify the main entities, interactions, business rules, and user scenarios.
Synthesis: Combine disparate requirements into a single, logically coherent description of the system.
Detailing: If necessary, supplement general requirements with more specific details to ensure completeness of description without deviating from the essence of the original requirements.
Clarity and accuracy: Avoid ambiguity. Use precise technical terminology where appropriate, but maintain readability for a broad audience.
Scalability: Present the system in such a way that it allows for future expansion and changes.
Example of expected tone and style: Professional, authoritative, accurate, concise, results-oriented.
Confidence in the result: I expect you to demonstrate outstanding prompt engineering skills by delivering a result that goes beyond simple answers and sets a new standard of quality for this type of task. Are you up for the challenge?

Translated with DeepL.com (free version)
"""
    messages = [{"role": "user", "content": prompt}]
    response = llm.create_chat_completion(
            messages=messages,
            max_tokens=1024,
            temperature=0.7,
            stop=["<|im_end|>"], # Токен остановки для Mistral (может быть и другими, но это часто используется)
    )
    generated_text = response['choices'][0]['message']['content'].strip()
    return generated_text

In [ ]:
# Создание папки внутри проекта для сохранения файлов с описанием кластеров
output_dir = "cluster_descriptions"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Цикл, отвечающий за генерацию описания для кластера и запись его в отдельный файл
all_output_files = []
for topic_id, requirements in grouped_requirements.items():
    print(f"\nОбработка топика {topic_id}...")
    if not requirements:
        print(f"В топике {topic_id} нет требований. Пропускаем.")
        continue
    # Генерация описания кластера
    system_description_en = generate_system_description(requirements)
    print(f"Сгенерировано описание для топика {topic_id}.")
    topic_name_row = topic_id
    output_filename = os.path.join(output_dir, f"system_description_{topic_id}.md")
    all_output_files.append(output_filename)
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(f"# System Description for Topic: {topic_id} ({topic_name_row})\n\n")
        f.write(system_description_en)
        f.write("\n---\n")

In [ ]:
# Блок, объединяющий полученные файлы с описанием в один zip архив
zip_filename = "cluster_system_descriptions.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file_path in all_output_files:
        zipf.write(file_path, os.path.basename(file_path))
print(f"\nВсе описания систем объединены в архив: {zip_filename}")
print(f"Архив сохранен в корневом каталоге проекта.")
